In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import zscore


In [3]:
# Load dataset
df = pd.read_csv('DSBDALExam DataSets/Adult/adult_dataset.csv')
df.head()


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
df_cleaned = df.replace(' ?', np.nan).dropna()

numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df_cleaned = df_cleaned[df_cleaned[col] >= 0]


In [5]:
z_scores = zscore(df_cleaned[['age', 'hours-per-week']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df_cleaned = df_cleaned[filtered_entries]


In [6]:
label_encoders = {}
for column in df_cleaned.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_cleaned[column] = le.fit_transform(df_cleaned[column])

X = df_cleaned.drop('income', axis=1)
y = df_cleaned['income']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [7]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)

nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_preds = nb_model.predict(X_test)

lr_acc = accuracy_score(y_test, lr_preds)
nb_acc = accuracy_score(y_test, nb_preds)
print("Logistic Regression Accuracy:", lr_acc)
print("Naïve Bayes Accuracy:", nb_acc)

print("\nLogistic Regression Report:\n", classification_report(y_test, lr_preds))
print("\nNaïve Bayes Report:\n", classification_report(y_test, nb_preds))


Logistic Regression Accuracy: 0.8250329929846496
Naïve Bayes Accuracy: 0.8036396471487115

Logistic Regression Report:
               precision    recall  f1-score   support

           0       0.85      0.94      0.89     11009
           1       0.70      0.46      0.55      3388

    accuracy                           0.83     14397
   macro avg       0.77      0.70      0.72     14397
weighted avg       0.81      0.83      0.81     14397


Naïve Bayes Report:
               precision    recall  f1-score   support

           0       0.82      0.95      0.88     11009
           1       0.66      0.34      0.45      3388

    accuracy                           0.80     14397
   macro avg       0.74      0.64      0.66     14397
weighted avg       0.79      0.80      0.78     14397

